In [ ]:
import camelot
import pandas as pd

tables = camelot.read_pdf("rhp.pdf", pages="all", flavor="stream")
financial_data = []


for table in tables:
    df = table.df
    df.columns = df.iloc[0]  
    df = df[1:]  
    financial_data.append(df)


financial_df = pd.concat(financial_data, ignore_index=True)

financial_df.columns = ["Metric", "FY2022", "FY2021", "FY2020"]
financial_df["FY2022"] = financial_df["FY2022"].apply(lambda x: float(x.replace(",", "")) if x else 0)


print(financial_df.head())

In [ ]:
financial_metrics = {
    "Net Profit Margin (%)": (financial_df.loc[financial_df["Metric"] == "Net Profit", "FY2022"].values[0] /
                              financial_df.loc[financial_df["Metric"] == "Revenue", "FY2022"].values[0]) * 100,
    "Current Ratio": (financial_df.loc[financial_df["Metric"] == "Current Assets", "FY2022"].values[0] /
                      financial_df.loc[financial_df["Metric"] == "Current Liabilities", "FY2022"].values[0]),
    "Debt-to-Equity Ratio": (financial_df.loc[financial_df["Metric"] == "Total Debt", "FY2022"].values[0] /
                             financial_df.loc[financial_df["Metric"] == "Total Equity", "FY2022"].values[0])
}


print("Financial Metrics:", financial_metrics)

In [ ]:
fin_metrics = {
    "Revenue Growth": (financial_df.loc[financial_df["Metric"] == "Revenue", "FY2022"].values[0] -
                       financial_df.loc[financial_df["Metric"] == "Revenue", "FY2021"].values[0]) / \
                       financial_df.loc[financial_df["Metric"] == "Revenue", "FY2021"].values[0] * 100,  
    "Net Profit Margin": financial_metrics["Net Profit Margin (%)"],  
    "Debt-to-Equity Ratio": financial_metrics["Debt-to-Equity Ratio"],  
    "Current Ratio": financial_metrics["Current Ratio"],  
}


print("RHP Data:", fin_metrics)

In [ ]:
import pandas as pd

peer_companies = ["comp1", "comp2", "comp3"]
company_financials = {
    "comp1": {
        "Revenue": 1000,
        "Net Profit": 80,
    },
    "comp2": {
        "Revenue": 1500,
        "Net Profit": 120,
    },
    "comp3": {
        "Revenue": 2000,
        "Net Profit": 150,
    },
}

peer_df = pd.DataFrame(company_financials).T
peer_df["Company"] = peer_df.index
peer_df.reset_index(drop=True, inplace=True)

print("Peer Financial Data:")
print(peer_df)

rhp_data = {
    "Revenue": 1800,
    "Net Profit": 130,
}

rhp_df = pd.DataFrame([rhp_data])

peer_means = peer_df[["Revenue", "Net Profit"]].mean()
peer_stds = peer_df[["Revenue", "Net Profit"]].std()

rhp_zscores = (rhp_df[["Revenue", "Net Profit"]] - peer_means) / peer_stds
print("\nRHP Z-Scores (compared to peers):")
print(rhp_zscores)

rhp_ranks = rhp_zscores.rank(ascending=False)
print("\nRHP Z-Scores and Ranks:")
print(rhp_ranks)

rhp_df["Company"] = "RHP Company"
peer_df = pd.concat([peer_df, rhp_df], ignore_index=True)

print("\nPeer and RHP Comparison:")
print(peer_df[["Company", "Revenue", "Net Profit"]])

In [ ]:
import yfinance as yf
import pandas as pd

peer_companies = ["comp1", "comp2", "comp3"] 
financial_data = []

company_financials = {
    "comp1": {
        "Revenue Growth": 15.0,  
        "Net Profit Margin": 8.0,
        "Debt-to-Equity Ratio": 0.5,
        "Current Ratio": 1.8,
    },
    "comp2": {
        "Revenue Growth": 12.5,
        "Net Profit Margin": 7.5,
        "Debt-to-Equity Ratio": 0.4,
        "Current Ratio": 2.0,
    },
    "comp3": {
        "Revenue Growth": 18.0,
        "Net Profit Margin": 9.0,
        "Debt-to-Equity Ratio": 0.6,
        "Current Ratio": 2.1,
    },
}

\
for company in peer_companies:
    stock = yf.Ticker(company)
    history = stock.history(period="2y") 

    # Calculate Stock Growth and Volatility
    stock_growth = ((history["Close"].iloc[-1] - history["Close"].iloc[0]) / history["Close"].iloc[0]) * 100
    stock_volatility = history["Close"].std()  

    financial_metrics = {
        "Company": company,
        "Revenue Growth": company_financials[company]["Revenue Growth"],
        "Net Profit Margin": company_financials[company]["Net Profit Margin"],
        "Debt-to-Equity Ratio": company_financials[company]["Debt-to-Equity Ratio"],
        "Current Ratio": company_financials[company]["Current Ratio"],
        "Stock Growth": stock_growth,
        "Stock Volatility": stock_volatility,
    }
    
    financial_data.append(financial_metrics)

peer_df = pd.DataFrame(financial_data)

print(peer_df)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


X = peer_df[["Revenue Growth", "Net Profit Margin", "Debt-to-Equity Ratio", "Current Ratio"]]
y = peer_df["Stock Growth"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


reg_model = LinearRegression()
reg_model.fit(X_train, y_train)


y_pred = reg_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


rhp_data = {
    "Revenue Growth": fin_metrics["Revenue Growth"],  
    "Net Profit Margin": fin_metrics["Net Profit Margin"],  
    "Debt-to-Equity Ratio": fin_metrics["Debt-to-Equity Ratio"],  
    "Current Ratio": fin_metrics["Current Ratio"],  
}


rhp_df = pd.DataFrame([rhp_data])


predicted_growth = reg_model.predict(rhp_df)
print(f"Predicted Stock Growth for RHP Company: {predicted_growth[0]}%")

In [ ]:
# Normalize the predicted stock growth to a score
financial_health_score = min(max((predicted_growth[0] / 100) * 100, 0), 100)
print(f"Financial Health Score: {financial_health_score}")

weights = {"Company Ratios": 0.4, "Peer Comparison": 0.4, "Stock Performance": 0.2}


financial_health_score = (
    weights["Company Ratios"] * sum(financial_metrics.values()) +
    weights["Peer Comparison"] * financial_df[["Revenue_Rank", "NetProfit_Rank"]].mean().mean() +
    weights["Stock Performance"] * predicted_growth[0]
)

print("Final Financial Health Score:", financial_health_score)